In [0]:
import torch, os
import pandas as pd
import torch.nn as nn
from torch import optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
data = pd.read_csv("BMITrain.csv")

In [50]:
data

,Unnamed: 0,BPDiastolic,BPSystolic,CHF,ageGT50,hosp,maxBMI
0,ac174ae7-1214-4bc5-9156-c635e2126439,87.0,132.0,0.0,0.0,0.0,18.16
1,94825d13-fe19-429e-9d50-bb72c4e46d9b,121.0,196.0,0.0,1.0,1.0,30.43
2,9c861c04-8696-441b-b11c-7d9503ecdde7,88.0,139.0,0.0,1.0,1.0,30.06
3,8c6ddf8a-b4be-492e-b575-184c836c4f96,88.0,129.0,0.0,0.0,1.0,30.56
4,61f162b6-f583-4507-ba9f-b2fc13106d1d,87.0,131.0,0.0,0.0,1.0,21.35
...,...,...,...,...,...,...,...
1135,2804b06d-e7a7-4a93-a320-d69474109025,87.0,127.0,0.0,1.0,1.0,27.77
1136,a2727847-23ea-4c2e-86d4-5dc918b7bb37,89.0,132.0,0.0,0.0,1.0,17.49
1137,cf31dbbb-e63c-4a8f-b4b6-3fecff315a0f,84.0,138.0,0.0,0.0,0.0,28.01
1138,82004240-285f-42d7-8b87-1b56a0a54a5f,88.0,136.0,4.0,1.0,1.0,27.85


In [0]:
num_patients = data.shape[0]
train_df = data[:int(num_patients*0.5)]
test_df = data[int(num_patients*0.5):]
train_labels = train_df['hosp']
test_labels = test_df['hosp']
train_data = train_df.drop(columns="hosp")
train_data = train_df.drop(columns="Unnamed: 0")
test_data = test_df.drop(columns="hosp")
test_data = test_df.drop(columns="Unnamed: 0")

In [0]:
temp_tensor = torch.tensor(train_data.values)
label_tensor = torch.tensor(train_labels, dtype=torch.long)
test_tensor = torch.tensor(test_data.values)
test_label_tensor = torch.tensor(test_labels.values, dtype=torch.long)

In [0]:
train_data = [(temp_tensor[i], label_tensor[i]) for i in range(temp_tensor.size()[0])]
test_data = [(test_tensor[i], test_label_tensor[i]) for i in range(test_tensor.size()[0])]

In [0]:
class Brain(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        super(Brain, self).__init__()
        self.fc1= nn.Linear(input_size, hidden1_size)
        self.ReLU1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.ReLU2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
    
    def  forward(self,x):
        out = self.fc1(x)
        out = self.ReLU1(out)
        out = self.fc2(out)
        out = self.ReLU2(out)
        out = self.fc3(out)
        return out

In [55]:
model = Brain(6, 64, 32, 2)
model.double()
#lr = learning rate 
optimizer = optim.SGD(model.parameters(), lr = 0.001)
num_epochs = 30
print(model)

Brain(
  (fc1): Linear(in_features=6, out_features=64, bias=True)
  (ReLU1): ReLU()
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (ReLU2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)


In [0]:
batch_size = 30
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size = batch_size, shuffle = True)
#test_loader = torch.utils.data.DataLoader(dataset=testData, batch_size = batch_size, shuffle = True)
criterion = nn.CrossEntropyLoss()

In [57]:
for i in range(num_epochs):
    cum_loss = 0
    for inputs, labels in train_loader:
        # print(inputs)
        optimizer.zero_grad()
       # print(inputs.size())
        output = model(inputs)
#         output = output.long()
      #  print(output)
        loss = criterion(output, labels.long())
        loss.backward()
        optimizer.step()
        
        cum_loss += loss.item()
        
    print("Training loss: ", cum_loss / len(train_loader))

Training loss:  1.0900006945408958
Training loss:  0.7609092496251955
Training loss:  0.6508661278899437
Training loss:  0.6847429419770223
Training loss:  0.6540507350101431
Training loss:  0.6437021570293983
Training loss:  0.6433885254886672
Training loss:  0.6522913488347756
Training loss:  0.6643187619809778
Training loss:  0.640735991251198
Training loss:  0.5932598850733496
Training loss:  0.6032984341028599
Training loss:  0.5829715432192799
Training loss:  0.6261583121749313
Training loss:  0.6100364811889968
Training loss:  0.6125167730772397
Training loss:  0.5933416571198147
Training loss:  0.579904729900017
Training loss:  0.5933814072440359
Training loss:  0.6102990508646668
Training loss:  0.596419441826124
Training loss:  0.5840664288222651
Training loss:  0.5951678200678568
Training loss:  0.5712902151620829
Training loss:  0.5746385930807251
Training loss:  0.5789251626177034
Training loss:  0.5887023317110696
Training loss:  0.5870268516015947
Training loss:  0.58130

In [58]:
with torch.no_grad():
  numCorrect = 0
  numTotal = 0
  for item, label in train_data:
      prediction = model(item).max(0)[1].item()
      # if prediction == 0: print(prediction)
      if prediction == label:
        numCorrect += 1
      numTotal += 1
  print("Accuracy = ", (numCorrect / numTotal))
  print("Correct/Total:", numCorrect, "/", numTotal)

Accuracy =  0.7210526315789474
Correct/Total: 411 / 570
